In [31]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/salary-train.csv')

In [32]:
data.FullDescription = data.FullDescription.str.lower()

In [33]:
data.FullDescription = data.FullDescription.replace('[^a-zA-Z0-9]', ' ', regex=True)

In [34]:
data.LocationNormalized.fillna('nan', inplace=True)
data.ContractTime.fillna('nan', inplace=True)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=5)
full_desc = tfidf.fit_transform(data.FullDescription)

In [36]:
from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
# X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [37]:
import scipy.sparse as sp
features = sp.hstack([full_desc, X_train_categ])

In [40]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=1, random_state=241)
reg.fit(features, data.SalaryNormalized)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [41]:
test = pd.read_csv('data/salary-test-mini.csv')
test.FullDescription = test.FullDescription.str.lower()
test.FullDescription = test.FullDescription.replace('[^a-zA-Z0-9]', ' ', regex=True)
test_fd = tfidf.transform(test.FullDescription)
test_cat = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
test_feat = sp.hstack([test_fd, test_cat])

array([ 56555.61500155,  37188.32442618])

In [44]:
np.savetxt('output/week4/task1.txt', np.round(reg.predict(test_feat), 2), newline=' ', fmt='%.02f')